In [38]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import sys
import io

In [647]:
start = 2023
end = 2025 


end += 1

seasons_list = [year for year in range(start, end)]
seasons_list

[2023, 2024]

In [507]:
team_names = {
    'ATL': 'Atlanta Hawks',
    'BOS': 'Boston Celtics',
    'BRK': 'Brooklyn Nets',
    'CHO': 'Charlotte Hornets',
    'CHI': 'Chicago Bulls',
    'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks',
    'DEN': 'Denver Nuggets',
    'DET': 'Detroit Pistons',
    'GSW': 'Golden State Warriors',
    'HOU': 'Houston Rockets',
    'IND': 'Indiana Pacers',
    'LAC': 'Los Angeles Clippers',
    'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat',
    'MIL': 'Milwaukee Bucks',
    'MIN': 'Minnesota Timberwolves',
    'NOP': 'New Orleans Pelicans',
    'NYK': 'New York Knicks',
    'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic',
    'PHI': 'Philadelphia 76ers',
    'PHO': 'Phoenix Suns',
    'POR': 'Portland Trail Blazers',
    'SAC': 'Sacramento Kings',
    'SAS': 'San Antonio Spurs',
    'TOR': 'Toronto Raptors',
    'UTA': 'Utah Jazz',
    'WAS': 'Washington Wizards'
}

In [489]:
# team_names = {
#     # 'DET': 'Detroit Pistons',
#     # 'BOS': 'Boston Celtics',
#     # 'PHO': 'Phoenix Suns',
#     'MEM': 'Memphis Grizzlies',

# }

### Collect Roster Average Stats Per Season

In [139]:
# Convert height to inches (Ht format 'ft-in' to inches)
def height_to_inches(height):
    if pd.isna(height):
        return None
    feet, inches = map(int, height.split('-'))
    return feet * 12 + inches

In [183]:
def clean_roster(roster, team_names):

    roster['ht'] = roster['Ht'].apply(height_to_inches)
    
    roster['birth_year'] = pd.to_datetime(roster['Birth Date']).dt.year

    roster['age'] = roster['Season']- roster['birth_year'] -1

    roster['exp'] = roster['Exp'].replace('R', 0)

    roster['exp'] = roster['exp'].round().astype(int)

    roster['team'] = team_names

    roster.drop(columns=['College','No.','Player','Pos','Birth Date','Birth','birth_year'], inplace=True)

    return roster

In [444]:
# https://www.basketball-reference.com/teams/DAL/2025.html

def scrape_roster(season,team_names):
    all_data = pd.DataFrame()

    for team in team_names:

        url = f"https://www.basketball-reference.com/teams/{team}/{season}.html"

        response = requests.get(url)

        # print(response)
        # print(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find('table', {'id': 'roster'})

            if table:
                df = pd.read_html(io.StringIO(str(table)))[0]

                df['Season'] = season
                
                df = clean_roster(df,team)

                all_data = pd.concat([all_data, df], ignore_index=True)

            else:
            
                print(f"No table found for {season}")

        else:
            print(f"Failed to retrieve data for {season}")

        time.sleep(4)

    return all_data

In [ ]:
roster = scrape_roster(2025,team_names)

In [186]:
roster.sample(5)

,Ht,Wt,Exp,Season,ht,age,exp,team
294,6-5,215.0,4,2025,77,26,4,MEM
212,6-11,256.0,3,2025,83,29,3,HOU
47,6-5,185.0,1,2025,77,20,1,BRK
94,6-7,221.0,7,2025,79,34,7,CHI
116,6-7,230.0,8,2025,79,31,8,CLE


In [474]:
def team_avg_roster(roster):

    avg_roster = roster.groupby(['team', 'Season']).agg(
        avg_age=('age', 'mean'),
        avg_exp=('exp', 'mean'),
        avg_weight=('Wt', 'mean'),
        avg_height=('ht', 'mean')
    ).reset_index()


    avg_roster['team_full_name'] = avg_roster['team'].map(team_names)

    avg_roster = avg_roster[['Season','team', 'team_full_name', 'avg_age', 'avg_exp', 'avg_weight', 'avg_height']]
    
    return avg_roster

In [196]:
avg_roster = team_avg_roster(roster)

In [198]:
avg_roster.sample(5)

,Season,team,team_full_name,avg_age,avg_exp,avg_weight,avg_height
9,2025,GSW,Golden State Warriors,26.714286,4.761905,213.000000,78.238095
20,2025,OKC,Oklahoma City Thunder,24.000000,2.812500,204.833333,78.500000
28,2025,UTA,Utah Jazz,24.722222,3.777778,220.500000,78.833333
29,2025,WAS,Washington Wizards,24.941176,3.941176,219.500000,79.470588
0,2025,ATL,Atlanta Hawks,25.333333,3.523810,215.947368,79.000000


### Append pre-season odds

In [126]:
# https://www.basketball-reference.com/leagues/NBA_2025_preseason_odds.html

def scrape_preseason_odds(season):
    all_data = pd.DataFrame()


    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_preseason_odds.html"

    response = requests.get(url)

    # print(response)
    # print(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'NBA_preseason_odds'})

        if table:
            df = pd.read_html(io.StringIO(str(table)))[0]

            all_data = pd.concat([all_data, df], ignore_index=True)

        else:
        
            print(f"No table found for {season}")

    else:
        print(f"Failed to retrieve data for {season}")

    return all_data

In [ ]:
preseason_odds = scrape_preseason_odds(2025)

In [205]:
avg_n_odds = pd.merge(avg_roster, preseason_odds, left_on='team_full_name', right_on='Team', how='left')

avg_n_odds.drop(columns='Team', inplace=True)


In [206]:
avg_n_odds.sample(5)

,Season,team,team_full_name,avg_age,avg_exp,avg_weight,avg_height,Odds
26,2025,SAS,San Antonio Spurs,24.300000,3.800000,207.000000,78.550000,10000
8,2025,DET,Detroit Pistons,24.444444,3.055556,209.400000,78.555556,100000
24,2025,POR,Portland Trail Blazers,24.400000,3.050000,208.736842,79.300000,100000
12,2025,LAC,Los Angeles Clippers,27.000000,5.666667,216.470588,77.952381,3000
13,2025,LAL,Los Angeles Lakers,25.700000,4.400000,216.941176,79.400000,3000


### Append salary stats

In [490]:
def clean_salaries(salaries):
    
    salaries = salaries.head(25)

    salaries.rename(columns={"Unnamed: 1":"player_name"}, inplace=True)
    
    salaries['Salary'] = salaries['Salary'].fillna(0)

    salaries['Salary'] = salaries['Salary'].replace({'\$': '', ',': ''}, regex=True).astype(int)

    salaries = salaries.sort_values(by='Salary', ascending=False)

    salaries = salaries.drop(columns=['Rk'])

    return salaries

<>:9: SyntaxWarning: invalid escape sequence '\$'
<>:9: SyntaxWarning: invalid escape sequence '\$'
/var/folders/_8/rdftlf6j3n5dq8s2vsk7f2wh0000gn/T/ipykernel_66760/1335304924.py:9: SyntaxWarning: invalid escape sequence '\$'
  salaries['Salary'] = salaries['Salary'].replace({'\$': '', ',': ''}, regex=True).astype(int)


In [491]:
# https://www.basketball-reference.com/teams/CHO/2025.html

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def scrape_salaries(
        season,
        team_names,
        driver,
    ):

    all_data = pd.DataFrame()

    for team in team_names:

        url = f"https://www.basketball-reference.com/teams/{team}/{season}.html"

        driver.get(url)

        time.sleep(5)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        div = soup.find('div', {'id': 'div_salaries2'})
        
        if div:
            table = div.find('table', {'id': 'salaries2'})
            if table:
                df = pd.read_html(io.StringIO(str(table)))[0]
                df['Season'] = season
                df['team']= team
                df = clean_salaries(df)
                all_data = pd.concat([all_data, df], ignore_index=True)
            else:
                print(f"No table found inside the div for {team} in {season}")
        else:
            print(f"No div with id 'div_salaries2' found for {team} in {season}")
            
    return all_data

In [ ]:
driver = webdriver.Chrome() 

salaries = scrape_salaries(
    2024, 
    team_names,
    driver,
)

driver.quit()

#### Finding the number of top players per team per season

In [497]:
def count_top_players(salaries,threshold=20):
    # based on the teams salaries we are trying to identify the numbers of stars/top players on the team (a good example is the Phoenix Suns in 2025)

    above = salaries[salaries['salary_percentage'] > threshold].shape[0]
    below = salaries[salaries['salary_percentage'] <= threshold].shape[0]
    
    return pd.Series({'top_players': above, 'not_top_players': below})

In [498]:
def find_top_players(salaries):
    total_salary = salaries.groupby(['team', 'Season'])['Salary'].sum().reset_index(name='total_salary')

    salaries = salaries.merge(total_salary, on=['team', 'Season'], how='left')

    salaries['salary_percentage'] = (salaries['Salary'] / salaries['total_salary']) * 100

    top_players = salaries.groupby(['team', 'Season']).apply(count_top_players).reset_index()
    
    return top_players

In [435]:
salaries[salaries['team']=='LAL'].sort_values('salary_percentage', ascending=False).head(20)

,player_name,Salary,Season,team,total_salary,salary_percentage
194,LeBron James,48728845,2025,LAL,187028846,26.054187
195,Anthony Davis,43219440,2025,LAL,187028846,23.108435
196,D'Angelo Russell,18692307,2025,LAL,187028846,9.994344
197,Rui Hachimura,17000000,2025,LAL,187028846,9.089507
198,Austin Reaves,12976362,2025,LAL,187028846,6.938161
199,Gabe Vincent,11000000,2025,LAL,187028846,5.881446
200,Jarred Vanderbilt,10714286,2025,LAL,187028846,5.728681
201,Max Christie,7142857,2025,LAL,187028846,3.819121
202,Jalen Hood-Schifino,3879840,2025,LAL,187028846,2.074461
203,Dalton Knecht,3819120,2025,LAL,187028846,2.041995


In [430]:
team_salary_stats = salaries.groupby(['team', 'Season']).agg(
    highest_salary=('Salary', 'max'),
    median_salary=('Salary', 'median'),
    total_salary=('Salary', 'sum'),
    
).reset_index()

top_players = find_top_players(salaries)

team_salary_stats = pd.merge(team_salary_stats, top_players, on=['team', 'Season'], how="left")

In [431]:
team_salary_stats.sample(3)

,team,Season,highest_salary,median_salary,total_salary,top_players,not_top_players
10,HOU,2025,42846615,9249960.0,165026987,1,14
29,WAS,2025,29651786,6692959.0,158877689,0,16
18,NOP,2025,36725670,5159854.0,163702985,3,8


In [433]:
avg_odds_salary_players = pd.merge(avg_n_odds, team_salary_stats,on=['team', 'Season'], how='left')


In [437]:
avg_odds_salary_players.sample(10)

,Season,team,team_full_name,avg_age,avg_exp,avg_weight,avg_height,Odds,highest_salary,median_salary,total_salary,top_players,not_top_players
2,2025,BRK,Brooklyn Nets,24.947368,3.947368,214.111111,79.052632,100000,40338144,4041249.0,165717974,1,14
5,2025,CLE,Cleveland Cavaliers,25.750000,4.200000,208.263158,77.950000,5000,36725670,8500000.0,161102772,2,11
15,2025,MIA,Miami Heat,26.700000,5.050000,218.750000,78.000000,4000,48798677,4655040.0,184058374,1,12
18,2025,NOP,New Orleans Pelicans,25.750000,3.150000,210.941176,78.400000,5000,36725670,5159854.0,163702985,3,8
29,2025,WAS,Washington Wizards,24.941176,3.941176,219.500000,79.470588,100000,29651786,6692959.0,158877689,0,16
17,2025,MIN,Minnesota Timberwolves,26.210526,4.578947,220.533333,79.263158,850,49205800,5287710.0,205560682,3,11
1,2025,BOS,Boston Celtics,26.318182,4.727273,220.315789,79.318182,300,49700000,3256341.5,201620656,1,15
25,2025,SAC,Sacramento Kings,26.000000,4.611111,212.071429,78.388889,6600,40500000,8000000.0,167640684,2,11
14,2025,MEM,Memphis Grizzlies,25.050000,3.150000,213.266667,78.900000,4000,36725670,4858705.5,177014145,1,15
20,2025,OKC,Oklahoma City Thunder,24.000000,2.812500,204.833333,78.500000,900,35859950,6669000.0,159241956,1,14


In [436]:
avg_odds_salary_players.to_csv('data/2025_team_data.csv')

### Append Franchise History/Championship stats

In [606]:
def clean_champions(df):

    nba_champions = df[[('Unnamed: 0_level_0', 'Year'), ('Finals', 'Champion')]]
    nba_champions.columns = ['Year', 'Champion']

    nba_champions = nba_champions[nba_champions.Year.notna()]

    nba_champions.Year = nba_champions.Year.astype(int)

    return nba_champions

In [607]:
# https://www.basketball-reference.com/playoffs/

def scrape_champions():
    df = pd.DataFrame()

    url = f"https://www.basketball-reference.com/playoffs/"

    response = requests.get(url)

    # print(response)
    # print(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'champions_index'})

        if table:
            df = pd.read_html(io.StringIO(str(table)))[0]
            df = clean_champions(df)
        else:
        
            print(f"No table found ")

    else:
        print(f"Failed to retrieve data ")

    return df

In [667]:
# Count number of championships for a given team and season

def nb_championships(team_names, season):

    nba_champions = scrape_champions()

    rev_team_names = {v: k for k, v in team_names.items()}

    nba_champions['team'] = nba_champions['Champion'].map(rev_team_names)
    
    team_championships = []
    
    for team in team_names:
        nb = len(nba_champions[(nba_champions['team'] == team) & (nba_champions['Year'] <= season)])
        
        team_championships.append({
            'team': team,
            'Season': season,
            'nb_championships': nb
        })
    
    df = pd.DataFrame(team_championships)
    
    return df


In [668]:
team_championships = nb_championships(team_names,2025)

avg_odds_salary_players_champ = avg_odds_salary_players.merge(team_championships, on=['team', 'Season'], how='left')

In [684]:
avg_odds_salary_players_champ.tail(5)

,Season,team,team_full_name,avg_age,avg_exp,avg_weight,avg_height,Odds,highest_salary,median_salary,total_salary,top_players,not_top_players,nb_championships
25,2025,SAC,Sacramento Kings,26.000000,4.611111,212.071429,78.388889,6600,40500000,8000000.0,167640684,2,11,0
26,2025,SAS,San Antonio Spurs,24.300000,3.800000,207.000000,78.550000,10000,29347826,9104643.5,142918889,1,13,4
27,2025,TOR,Toronto Raptors,25.368421,3.631579,210.615385,78.210526,50000,32500000,10130980.0,155718137,1,12,1
28,2025,UTA,Utah Jazz,24.722222,3.777778,220.500000,78.833333,50000,42176400,5469120.0,128678582,2,9,0
29,2025,WAS,Washington Wizards,24.941176,3.941176,219.500000,79.470588,100000,29651786,6692959.0,158877689,0,16,0


### Loop for multiple seasons :o

In [ ]:
def scrape_all_rosters():
    all_rosters = pd.DataFrame()

    for season in range(start, end):  
        print(f"Scraping data for the {season-1}-{season} season...")

        roster = scrape_roster(season,team_names)

        if roster is not None:
            all_rosters = pd.concat([all_rosters, roster], ignore_index=True)

    all_avg_roster = team_avg_roster(all_rosters)
    
    return all_avg_roster

all_avg_roster = scrape_all_rosters()

In [476]:
all_avg_roster.to_csv(f"data/{start}_{end-1}_avg_roster.csv")

In [ ]:
def scrape_all_preseason_odds():
    all_preseason_odds = pd.DataFrame()

    for season in range(start, end):  
        print(f"Scraping data for the {season-1}-{season} season...")

        preseason_odds = scrape_preseason_odds(season,team_names)

        if preseason_odds is not None:
            all_preseason_odds = pd.concat([all_preseason_odds, preseason_odds], ignore_index=True)

    return all_preseason_odds

all_preseason_odds = scrape_all_preseason_odds()

all_avg_n_odds = pd.merge(avg_roster, preseason_odds, left_on='team_full_name', right_on='Team', how='left')

all_avg_n_odds.drop(columns='Team', inplace=True)


In [479]:
all_avg_n_odds.to_csv(f"data/{start}_{end-1}_avg_n_odds.csv")

In [ ]:
def scrape_all_salaries():
    all_salaries = pd.DataFrame()

    driver = webdriver.Chrome() 

    for season in range(start, end):  
        print(f"Scraping data for the {season-1}-{season} season...")

        salaries = scrape_salaries(
            season,
            team_names,
            driver,
        )

        if salaries is not None:
            all_salaries = pd.concat([all_salaries, salaries], ignore_index=True)

    driver.quit()
    
    return all_salaries

all_salaries = scrape_all_salaries()

In [509]:
all_salaries.to_csv(f"data/{start}_{end-1}_salaries_stats.csv")

In [533]:
all_top_players = find_top_players(all_salaries)

team_salary_stats = all_salaries.groupby(['team', 'Season']).agg(
    highest_salary=('Salary', 'max'),
    median_salary=('Salary', 'median'),
    total_salary=('Salary', 'sum'),
    
).reset_index()

all_team_salary_stats = pd.merge(team_salary_stats, all_top_players, on=['team', 'Season'], how="left")

/var/folders/_8/rdftlf6j3n5dq8s2vsk7f2wh0000gn/T/ipykernel_66760/2735718874.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_players = salaries.groupby(['team', 'Season']).apply(count_top_players).reset_index()


In [541]:
all_avg_odds_salary_players = pd.merge(all_avg_n_odds, all_team_salary_stats,on=['team', 'Season'], how='left')

In [543]:
all_avg_odds_salary_players.sample(10)

,Season,team,team_full_name,avg_age,avg_exp,avg_weight,avg_height,Odds,highest_salary,median_salary,total_salary,top_players,not_top_players
52,2023,SAS,San Antonio Spurs,23.652174,3.000000,218.130435,78.869565,10000,13750000,2600000.0,101438004,0,25
15,2024,DEN,Denver Nuggets,26.294118,4.882353,221.235294,79.117647,750,47607350,2367300.0,177890198,1,17
31,2024,MIA,Miami Heat,27.380952,5.666667,211.333333,78.000000,4000,45183960,2352000.0,174695832,1,20
29,2024,MEM,Memphis Grizzlies,24.969697,3.060606,214.939394,77.939394,4000,34005250,2400000.0,160019663,1,24
33,2024,MIL,Milwaukee Bucks,28.285714,6.714286,217.238095,78.380952,1100,45640084,2019706.0,182226590,2,18
49,2024,POR,Portland Trail Blazers,24.272727,2.318182,215.181818,78.909091,100000,32459438,1951220.0,165919847,0,25
13,2024,DAL,Dallas Mavericks,26.000000,4.727273,215.500000,78.590909,1000,40064220,2541022.0,163631110,2,20
2,2023,BOS,Boston Celtics,26.722222,5.333333,225.388889,78.944444,300,30351780,3903140.5,175733265,0,18
37,2024,NOP,New Orleans Pelicans,25.200000,3.350000,212.500000,78.350000,5000,35802469,1836096.0,164937685,3,20
10,2023,CLE,Cleveland Cavaliers,26.944444,5.388889,220.500000,78.444444,5000,30913750,2000000.0,149251300,1,20


In [688]:
def all_nba_championships(team_names,seasons_list):
    
    all_team_championships = pd.DataFrame()

    for season in seasons_list:
        team_championships = nb_championships(team_names,season)
        
        all_team_championships = pd.concat([all_team_championships, team_championships], ignore_index=True)

    return all_team_championships

In [690]:
all_team_championships = all_nba_championships(team_names,seasons_list)

In [693]:
all_avg_odds_salary_players_champ = all_avg_odds_salary_players.merge(all_team_championships, on=['team', 'Season'], how='left')

In [696]:
all_avg_odds_salary_players_champ[all_avg_odds_salary_players_champ.team=='BOS']

,Season,team,team_full_name,avg_age,avg_exp,avg_weight,avg_height,Odds,highest_salary,median_salary,total_salary,top_players,not_top_players,nb_championships
2,2023,BOS,Boston Celtics,26.722222,5.333333,225.388889,78.944444,300,30351780,3903140.5,175733265,0,18,1
3,2024,BOS,Boston Celtics,25.894737,4.684211,216.631579,79.263158,300,36861707,2019706.0,183679244,1,20,2


In [698]:
all_avg_odds_salary_players.to_csv(f"data/{start}_{end-1}_avg_odds_salary_players_champ.csv")

### Other

In [ ]:
# def scrape_season(season):
#     all_data = pd.DataFrame()

#     url = f'https://www.basketball-reference.com/leagues/NBA_{season}.html'
#     response = requests.get(url)

#     if response.status_code == 200:
#         soup = BeautifulSoup(response.content, 'html.parser')
#         table = soup.find('table', {'id': 'totals-team'})

#         if table:
#             df = pd.read_html(io.StringIO(str(table)))[0]

#             df['Season'] = season

#             all_data = pd.concat([all_data, df], ignore_index=True)
#         else:
#             print(f"No table found for {season}")
#     else:
#         print(f"Failed to retrieve data for {season}")

#     time.sleep(4)

#     return all_data

# def scrape_all_seasons():
#     all_data = pd.DataFrame()

#     for season in range(start, end):  
#         data = scrape_season(str(season))

#         if data is not None:
#             data['Season'] = season
#             all_data = pd.concat([all_data, data], ignore_index=True)

#             print(f"Scraping data for season {season}")

#     return all_data

# result = scrape_all_seasons()

# result['make_playoffs'] = result['Team'].apply(lambda x: 1 if '*' in x else 0)
# result['Team'] = result['Team'].str.replace('*', '')

# result.head(10)


In [ ]:
# def scrape_season_table(season, table_id):
#     all_data = pd.DataFrame()

#     url = f'https://www.basketball-reference.com/awards/awards_{season}.html'

#     options = webdriver.ChromeOptions()
#     options.add_argument('--headless')  
#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
#     driver.get(url)

#     try:
#         table = driver.find_element(By.ID, table_id)
#         table_html = table.get_attribute('outerHTML')
#         driver.quit()

#         df = pd.read_html(StringIO(table_html), header=[1])[0]

#         df['Season'] = season
#         df['award_type'] = table_id

#         all_data = pd.concat([all_data, df], ignore_index=True)
#     except Exception as e:
#         print(f"No table found for {table_id} in {season}")
#         print(e)
#         driver.quit()
#         return None

#     time.sleep(5)

#     return all_data

# def scrape_all_seasons_tables(seasons, table_ids):
#     all_data = pd.DataFrame()

#     total_tasks = len(seasons) * len(table_ids)
#     task_count = 0

#     for season in reversed(seasons):
#         for table_id in table_ids:
#             data = scrape_season_table(season, table_id)

#             if data is not None:
#                 all_data = pd.concat([all_data, data], ignore_index=True)

#             task_count += 1
#             completion_percentage = (task_count / total_tasks) * 100
#             sys.stdout.write(f"\rScraping: [{'#' * int(completion_percentage // 2)}{' ' * (50 - int(completion_percentage // 2))}] {completion_percentage:.2f}%")
#             sys.stdout.flush()

#     sys.stdout.flush()

#     print("Columns before removing 'Unnamed':", all_data.columns)

#     all_data.columns = all_data.columns.map(str)
#     all_data = all_data.loc[:, ~all_data.columns.str.contains('^Unnamed')]

#     print("Columns after removing 'Unnamed':", all_data.columns)

#     if isinstance(all_data.columns, pd.MultiIndex):
#         all_data.columns = [' '.join(col).strip() for col in all_data.columns.values]

#     print("Columns after flattening MultiIndex:", all_data.columns)
    
#     return all_data

# seasons_list = [str(year) for year in range(start, end)]
# table_ids = ['mvp', 'roy', 'dpoy', 'smoy', 'mip', 'clutch_poy', 'leading_all_nba', 'leading_all_defense', 'leading_all_rookie', 'coy']

# result = scrape_all_seasons_tables(seasons_list, table_ids)

# all_award_voting = result

# #save
# all_award_voting.to_csv('all_award_voting.csv', index=False, encoding="utf-8-sig")